На основании нормативов по качеству питьевой воды и прайс листа с ценами бутилированной воды создайте нейро-консультанта, который бы отвечал на вопросы по данной информации.
Для этого скачайте pdf файлы по ссылкам:

**Нормативы**: https://files.stroyinf.ru/Data2/1/4294850/4294850590.pdf

https://files.stroyinf.ru/Data2/1/4294846/4294846957.pdf

**Прайс-лист**: https://xn---56-5cdablc8cp7g.xn--p1ai/ckfinder/userfiles/files/price%20ot%202019-01-01.pdf

Документы с нормативами объедините в 1 БЗ, а из прайс-листа сделайте вторую БЗ.

Нейро-консультант должен на основании вопроса пользователя динамически выбирать соответствующую базу знаний и отвечать на онове нее на вопрос.

Проверьте работу нейро-консультанта на самостоятельно-сформулированных вопросах.

In [ ]:
!pip install tiktoken==0.4.0 openai==0.28.0 langchain==0.0.281 unstructured pdf2image pdfminer.six faiss-cpu==1.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:0

In [ ]:
import tiktoken
import getpass
import openai
import tiktoken
from openai.embeddings_utils import distances_from_embeddings
import pandas as pd
import numpy as np
from ast import literal_eval
from langchain.chains.router import MultiRetrievalQAChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import os
from openai.embeddings_utils import distances_from_embeddings, cosine_similarity
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
def download_pdf_from_url(url, save_path):
    response = requests.get(url)

    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"PDF downloaded and saved as {save_path}")
    else:
        print(f"Failed to download PDF. HTTP Status Code: {response.status_code}")

url_1 = "https://files.stroyinf.ru/Data2/1/4294850/4294850590.pdf"
save_path_1 = "/content/water_control.pdf"
download_pdf_from_url(url_1, save_path_1)

url_2 = "https://files.stroyinf.ru/Data2/1/4294846/4294846957.pdf"
save_path_2 = "/content/sanpin.pdf"
download_pdf_from_url(url_2, save_path_2)

url_3 = "https://xn---56-5cdablc8cp7g.xn--p1ai/ckfinder/userfiles/files/price%20ot%202019-01-01.pdf"
save_path_3 = "/content/price.pdf"
download_pdf_from_url(url_3, save_path_3)

PDF downloaded and saved as /content/water_control.pdf
PDF downloaded and saved as /content/sanpin.pdf
PDF downloaded and saved as /content/price.pdf


In [ ]:
# функция, которая принимает список названий PDF-файлов и возвращает базу знаний FAISS
def create_faiss_vectorstore(pdf_names):
    pdf_folder_path = "/content/"

    # Фильтрация файлов: загрузка только файлов с определенными именами и расширением .pdf
    all_files = os.listdir(pdf_folder_path)
    pdf_files = [fn for fn in all_files if fn in pdf_names and fn.endswith('.pdf')]

    # Создание объектов UnstructuredPDFLoader для каждого PDF-файла
    loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in pdf_files]

    all_documents = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100, length_function=len)

    for loader in loaders:
        # Загрузка и разделение текста
        raw_documents = loader.load()
        documents = text_splitter.split_documents(raw_documents)
        all_documents.extend(documents)

    # Создание FAISS vectorstore
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(all_documents, OpenAIEmbeddings()).as_retriever()

    return vectorstore

In [ ]:
# Создаем БЗ по каждой спецификации (pdf)
pdf_name_norms = ['sanpin.pdf', 'water_control.pdf']
db_norms = create_faiss_vectorstore(pdf_name_norms)
pdf_name_price = ['price.pdf']
db_price = create_faiss_vectorstore(pdf_name_price)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Добавляем информацию и описание для наших векторных хранилищ (баз знаний)
retriever_infos = [
    {
        "name": "Normativi",
        "description": "Применяется для ответов на вопросы по гигиенические требованиям к качеству воды централизованных систем питьевого водоснабжения и по требованиям к организации и методам контроля качествакачества питьевой воды",
       "retriever": db_norms
    },
    {
        "name": "Price-List",
        "description": "Применяется для ответов на вопросы о стоимости питьевой воды и аксесуаров для воды",
        "retriever": db_price
    }
]

In [ ]:
chain = MultiRetrievalQAChain.from_retrievers(ChatOpenAI(temperature=0), retriever_infos, verbose=True)

In [ ]:
print(chain.run("Сколько стоит ручной насос для воды?"))



> Entering new MultiRetrievalQAChain chain...
Price-List: {'query': 'Сколько стоит ручной насос для воды?'}
> Finished chain.
Ручной насос для воды стоит 400 рублей.


In [ ]:
print(chain.run("Какие бывают методы определения обобщенных показателей качества питьевой воды?"))



> Entering new MultiRetrievalQAChain chain...
Normativi: {'query': 'Какие методы используются для определения обобщенных показателей качества питьевой воды?'}
> Finished chain.
Для определения обобщенных показателей качества питьевой воды используются методы, указанные в таблице 2 ГОСТ Р 51232-98. Однако, конкретные методы определения не указаны в предоставленном контексте.


In [ ]:
print(chain.run("Опиши кратко порядок составления рабочей программы производственного контроля качества питьевой воды?"))



> Entering new MultiRetrievalQAChain chain...
Normativi: {'query': 'Опиши порядок составления рабочей программы производственного контроля качества питьевой воды'}
> Finished chain.
Порядок составления рабочей программы производственного контроля качества питьевой воды определяется в соответствии с санитарными правилами и нормами, утвержденными в установленном порядке.

Индивидуальные предприниматели и юридические лица, осуществляющие эксплуатацию системы водоснабжения, должны разработать рабочую программу на основе этих санитарных правил.

Рабочая программа должна включать в себя следующие элементы:

1. Определение контролируемых показателей: в рамках программы должны быть указаны показатели, которые будут контролироваться для обеспечения качества питьевой воды.

2. Определение количества и периодичности отбора проб: программа должна указывать, сколько проб питьевой воды будет отбираться для контроля и с какой периодичностью.

3. Организация производственного контроля: программа дол